# Benford for 3D MRI - Model training with cross validation

This script trains several machine learning regression models to estimate the level of Rician noise from the divergences of the observed probability distributions of the first and second digits of the transformed versions of noisy 3D magnetic resonance images (MRIs) with respect to the theoretical Benford probability distributions of those digits.

We focus on the distributions of the first digits because the distributions of the second digits do not seem to contain much useful information.

In order to keep things simple, we have tried all sets of input features for the regression models with one and two input features.

We apply 10-fold cross validation. Samples from the same image are strongly correlated so they cannot be used in the training and validation sets.

In [1]:
%reset

In [16]:
# Import the relevant libraries
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy import stats
import os
from glob import glob 
import pickle

In [21]:
# Define the dataset to use, choosing one of the directories: 20OASIS_SynapseWeb_brain, OASIS-TRT-20_volumes, 12HLN, NKI-TRT-20_volumes, NKI-RS-22_volumes, MMRR-21_volumes
repository = '12HLN' # MUST TO DEFINE: #20OASIS_SynapseWeb_brain #OASIS-TRT-20_volumes #12HLN #NKI-TRT-20_volumes #NKI-RS-22_volumes #MMRR-21_volumes
typeImage = 't1weighted.nii.gz' #'t1weighted_brain.nii.gz' #'t1weighted.nii.gz'
inputDir = ('../input') 
outputDir = ('../output')

# Path with input images
dataMRI = os.path.join(inputDir, repository)
pathMRI = sorted(glob(dataMRI + '/**/' + typeImage, recursive=True))
SIZE = len(pathMRI)

# Training dataset path
str_path = os.path.join(outputDir + '/input/' + repository + '_head.pkl') 
dataset_path = str_path.replace("_volumes", "")

# Result folder path
str_path = os.path.join(outputDir + '/Figures_and_tables/' + repository)
result_path = str_path.replace("volumes", "head")

# Experiment parameters
NumNoiseLevels=20
NumNoiselessImages=SIZE # SIZE is the number of images in the repository: HLN=12; MMRR=21; NKI-RS=22; NKI-TRT=20; OASIS-TRT=20
MaxNoiseLevel=0.4
TestedTransforms=["FFT","DCT","DST"]
TestedPerformanceMeasures=["MSE","MAE","R2"]
TestedRegressors=["Linear","Poly","RandomForest","SVR","KernelRegression"]
NumSplits=10

In [23]:
# Load the training data file
with open(dataset_path,'rb') as MyFile:
  [NoiseLevels,Divergences]=pickle.load(MyFile)

In [ ]:
# Import machine learning regression models
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from statsmodels.nonparametric.kernel_regression import KernelReg
from sklearn.metrics import r2_score

# Import cross validation
from sklearn.model_selection import KFold

# Import linear algebra error
from numpy.linalg import LinAlgError

# Define the input features that will be considered. Each input feature is defined
# by a tuple that indicates the transform, the probability distribution divergence
# measure, and the digit (first or second)
InputFeatures=[("FFT","BD","First"),("DCT","BD","First"),("DST","BD","First"),
               ("FFT","KL","First"),("DCT","KL","First"),("DST","KL","First"),
               ("FFT","TV","First"),("DCT","TV","First"),("DST","TV","First"),
               ("FFT","H","First"),("DCT","H","First"),("DST","H","First"),
               ("FFT","JS","First"),("DCT","JS","First"),("DST","JS","First"),
               ("FFT","BD","Second"),("DCT","BD","Second"),("DST","BD","Second"),
               ("FFT","KL","Second"),("DCT","KL","Second"),("DST","KL","Second"),
               ("FFT","TV","Second"),("DCT","TV","Second"),("DST","TV","Second"),
               ("FFT","H","Second"),("DCT","H","Second"),("DST","H","Second"),
               ("FFT","JS","Second"),("DCT","JS","Second"),("DST","JS","Second")
               ]

# Define the sets of model input features to be tried. Each set is defined by 
# a list of input features which are indices into the InputFeatures list.
# The sets [0] and [3] were presented in the MICCAI paper. All the others are new.
CountInputFeatures=15
SetsInputFeatures=[]
for NdxFeature in range(0,CountInputFeatures):
  SetsInputFeatures.append([NdxFeature])
for NdxFeature in range(0,CountInputFeatures):
  for NdxFeature2 in range(NdxFeature+1,CountInputFeatures):
    SetsInputFeatures.append([NdxFeature,NdxFeature2])
# These additional sets of features perform fairly well
SetsInputFeatures.append([3,10,13])
SetsInputFeatures.append([3,4,5])
SetsInputFeatures.append([3,4,5,8])
# The number of sets of inputs features, to be used afterwards
NumSets=len(SetsInputFeatures)

# Create a name for each set of input features. Each name is a string with the input
# feature indices separated by blank spaces
SetNames=[]
for NdxSet in range(0,NumSets):
  SetNames.append(" ".join(map(str,SetsInputFeatures[NdxSet])))

# Create the cross validation class to obtain the image indices for the training
# and validation sets
CrossValidator = KFold(n_splits=NumSplits, shuffle=True, random_state=1)

# Prepare the validation results output variable
ValidationResults={}

for NdxFold, (TrainImagesIndices, ValidationImagesIndices) in enumerate(CrossValidator.split(np.arange(NumNoiselessImages))):
  print(f"Fold {NdxFold}:")
  print(f"  Train: index={TrainImagesIndices}")
  print(f"  Test:  index={ValidationImagesIndices}")

  # Obtain the training noise levels (outputs to be predicted by the regressors) from
  # the training noiseless images 
  TrainingNoiseLevels=NoiseLevels[TrainImagesIndices,:]
  # Reshape to the (n_samples, n_targets) format required by
  # the sklearn.linear_model.LinearRegression class
  TrainingNoiseLevelsFlat=np.reshape(TrainingNoiseLevels,(len(TrainImagesIndices)*NumNoiseLevels,1))

  # Obtain the validation noise levels (outputs to be predicted by the regressors) from
  # the validation noiseless images
  ValidationNoiseLevels=NoiseLevels[ValidationImagesIndices,:]
  # Reshape to the (n_samples, n_targets) format required by
  # the sklearn.linear_model.LinearRegression class
  ValidationNoiseLevelsFlat=np.reshape(ValidationNoiseLevels,(len(ValidationImagesIndices)*NumNoiseLevels,1))



  # Train the models and validate their performance
  for NdxSet in range(0,NumSets):

    # Print the variable indices for the current model
    print(SetsInputFeatures[NdxSet])

    # Obtain the number of input features (distribution distances) for the current model
    NumInputFeatures=len(SetsInputFeatures[NdxSet])
    
    # Obtain the input features (distribution distances) for the training and validation sets
    TrainingDistributionDistances=np.zeros((NumInputFeatures,len(TrainImagesIndices),NumNoiseLevels))
    ValidationDistributionDistances=np.zeros((NumInputFeatures,len(ValidationImagesIndices),NumNoiseLevels))
    # Loop for all input features in the current set
    for NdxInputFeature in range(0,NumInputFeatures):
      # Fetch the input features for all noiseless images except the validation noiseless image
      TrainingDistributionDistances[NdxInputFeature,:,:]=Divergences[InputFeatures[SetsInputFeatures[NdxSet][NdxInputFeature]]][TrainImagesIndices,:]
      # Fetch the input features for the validation noiseless image
      ValidationDistributionDistances[NdxInputFeature,:,:]=Divergences[InputFeatures[SetsInputFeatures[NdxSet][NdxInputFeature]]][ValidationImagesIndices,:]

    # Reshape the input features to the format (n_samples, n_features) which is required
    # by the sklearn.linear_model.LinearRegression class
    TrainingDistributionDistancesFlat=np.reshape(TrainingDistributionDistances,(NumInputFeatures,len(TrainImagesIndices)*NumNoiseLevels)).transpose()
    ValidationDistributionDistancesFlat=np.reshape(ValidationDistributionDistances,(NumInputFeatures,len(ValidationImagesIndices)*NumNoiseLevels)).transpose()

    # Train the linear regression model
    LinearModel1D=LinearRegression().fit(TrainingDistributionDistancesFlat,TrainingNoiseLevelsFlat)

    # Train the polynomial regression model
    poly = PolynomialFeatures(degree=2, include_bias=False)
    TrainingPolyFeatures = poly.fit_transform(TrainingDistributionDistancesFlat)
    PolyModel1D = LinearRegression()
    PolyModel1D.fit(TrainingPolyFeatures, TrainingNoiseLevelsFlat)

    # Train the random forest regression model
    RandomForestModel=RandomForestRegressor(min_samples_leaf=20).fit(TrainingDistributionDistancesFlat,TrainingNoiseLevelsFlat.ravel())

    # Train the support vector regression model
    SVRModel = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.001)).fit(TrainingDistributionDistancesFlat,TrainingNoiseLevelsFlat.ravel())

    # Train the kernel regression model
    try:
      KRModel = KernelReg(endog=TrainingNoiseLevelsFlat.ravel(), exog=TrainingDistributionDistancesFlat, var_type="c"*NumInputFeatures, bw="cv_ls") 
    except LinAlgError:
      print("Linear algebra error while training the kernel regressor")

    # Validate the linear regression model
    ValidationNoiseLevelsLinear=LinearModel1D.predict(ValidationDistributionDistancesFlat)
    ValidationResults[NdxFold,NdxSet,"MSE","Linear"]=np.mean(np.power(ValidationNoiseLevelsLinear-ValidationNoiseLevelsFlat,2))
    ValidationResults[NdxFold,NdxSet,"MAE","Linear"]=np.mean(np.absolute(ValidationNoiseLevelsLinear-ValidationNoiseLevelsFlat))
    ValidationResults[NdxFold,NdxSet,"R2","Linear"]=r2_score(ValidationNoiseLevelsFlat,ValidationNoiseLevelsLinear)

    # Validate the polynomial regression model
    ValidationPolyFeatures = poly.fit_transform(ValidationDistributionDistancesFlat)
    ValidationNoiseLevelsPoly=PolyModel1D.predict(ValidationPolyFeatures)
    ValidationResults[NdxFold,NdxSet,"MSE","Poly"]=np.mean(np.power(ValidationNoiseLevelsPoly-ValidationNoiseLevelsFlat,2))
    ValidationResults[NdxFold,NdxSet,"MAE","Poly"]=np.mean(np.absolute(ValidationNoiseLevelsPoly-ValidationNoiseLevelsFlat))
    ValidationResults[NdxFold,NdxSet,"R2","Poly"]=r2_score(ValidationNoiseLevelsFlat,ValidationNoiseLevelsPoly)  

    # Validate the random forest regression model
    ValidationNoiseLevelsRandomForest=np.reshape(RandomForestModel.predict(ValidationDistributionDistancesFlat),(NumNoiseLevels*len(ValidationImagesIndices),1))
    ValidationResults[NdxFold,NdxSet,"MSE","RandomForest"]=np.mean(np.power(ValidationNoiseLevelsRandomForest-ValidationNoiseLevelsFlat,2))
    ValidationResults[NdxFold,NdxSet,"MAE","RandomForest"]=np.mean(np.absolute(ValidationNoiseLevelsRandomForest-ValidationNoiseLevelsFlat))
    ValidationResults[NdxFold,NdxSet,"R2","RandomForest"]=r2_score(ValidationNoiseLevelsFlat,ValidationNoiseLevelsRandomForest)  
    
    # Validate the support vector regression model
    ValidationNoiseLevelsSVR=np.reshape(SVRModel.predict(ValidationDistributionDistancesFlat),(NumNoiseLevels*len(ValidationImagesIndices),1))
    ValidationResults[NdxFold,NdxSet,"MSE","SVR"]=np.mean(np.power(ValidationNoiseLevelsSVR-ValidationNoiseLevelsFlat,2))
    ValidationResults[NdxFold,NdxSet,"MAE","SVR"]=np.mean(np.absolute(ValidationNoiseLevelsSVR-ValidationNoiseLevelsFlat))
    ValidationResults[NdxFold,NdxSet,"R2","SVR"]=r2_score(ValidationNoiseLevelsFlat,ValidationNoiseLevelsSVR)  

    # Validate the kernel regression model
    # Error to estimate hyperparameter
    try:
      (Averages,StandardDeviations)=KRModel.fit(ValidationDistributionDistancesFlat)
      ValidationNoiseLevelsKR=np.reshape(Averages,(NumNoiseLevels*len(ValidationImagesIndices),1))
      ValidationResults[NdxFold,NdxSet,"R2","KernelRegression"]=r2_score(ValidationNoiseLevelsFlat,ValidationNoiseLevelsKR)  
      ValidationResults[NdxFold,NdxSet,"MSE","KernelRegression"]=np.mean(np.power(ValidationNoiseLevelsKR-ValidationNoiseLevelsFlat,2))
      ValidationResults[NdxFold,NdxSet,"MAE","KernelRegression"]=np.mean(np.absolute(ValidationNoiseLevelsKR-ValidationNoiseLevelsFlat))
    except LinAlgError: # Error del 21MMRR, fallo en entrenamiento, continua y hae el fit. Pero falla en reshape.
      print("Linear algebra error while training the kernel regressor so can not do reshape after fit.")
      ValidationResults[NdxFold,NdxSet,"R2","KernelRegression"]=r2_score(ValidationNoiseLevelsFlat,ValidationNoiseLevelsKR)   
      ValidationResults[NdxFold,NdxSet,"MSE","KernelRegression"]=np.mean(np.power(ValidationNoiseLevelsKR-ValidationNoiseLevelsFlat,2))
      ValidationResults[NdxFold,NdxSet,"MAE","KernelRegression"]=np.mean(np.absolute(ValidationNoiseLevelsKR-ValidationNoiseLevelsFlat))
    except ValueError: # Error del 22NKI, Nan en Average y da error en R2.
      print('Nan or Inf value in R2 because of to KernelModel small bandwith')
      ValidationResults[NdxFold,NdxSet,"MSE","KernelRegression"]=np.Inf
      ValidationResults[NdxFold,NdxSet,"MAE","KernelRegression"]=np.Inf
      ValidationResults[NdxFold,NdxSet,"R2","KernelRegression"]=0
    
    # Plot the regression results for the models with only one input feature
    if NumInputFeatures==1:
      # Sort the values of the independent variable for the plot (the distribution divergence values)
      SortedIndices = np.argsort(ValidationDistributionDistancesFlat.flatten())
      fig=plt.figure()
      plt.plot(TrainingDistributionDistancesFlat, TrainingNoiseLevelsFlat, 'o', ValidationDistributionDistancesFlat[SortedIndices], ValidationNoiseLevelsLinear[SortedIndices], '-',
              ValidationDistributionDistancesFlat[SortedIndices],ValidationNoiseLevelsPoly[SortedIndices],'-',ValidationDistributionDistancesFlat[SortedIndices],ValidationNoiseLevelsRandomForest[SortedIndices],'-',
              ValidationDistributionDistancesFlat[SortedIndices],ValidationNoiseLevelsSVR[SortedIndices],'-',ValidationDistributionDistancesFlat[SortedIndices],ValidationNoiseLevelsKR[SortedIndices],'-', markersize=1.3)
      plt.legend(['data', 'linear', 'poly', 'RF', 'SVR', 'KR'], loc='best')
      plt.xlabel('Mi') # MUST TO DEFINE: write the value of the metric with a number.
      plt.ylabel('Noise level')
      plot_path = os.path.join(result_path + '/Fold%s_Plot%s.pdf' % (NdxFold, SetNames[NdxSet]))
      plt.savefig(plot_path, format="pdf", bbox_inches="tight")
      #plt.show()
      plt.close(fig)

In [ ]:
# Write the results to an Excel spreadsheet file

# Average the validation results across the splits
ValidationResultsAverages={}
for NdxSet in range(0,NumSets):
  for MyPerformanceMeasure in TestedPerformanceMeasures:
    for MyRegressor in TestedRegressors:
      MyResults=np.zeros((NumSplits,1))
      for NdxFold in np.arange(NumSplits):
        MyResults[NdxFold]=ValidationResults[NdxFold,NdxSet,MyPerformanceMeasure,MyRegressor]
      ValidationResultsAverages[NdxSet,MyPerformanceMeasure,MyRegressor]=np.mean(MyResults)

# Import the library to write Excel spreadsheet files
import xlwt

# Define the performance measures and the regressors
HigherBetterPerformance={"MSE":False,"MAE":False,"R2":True}
TestedRegressorsSmall=["Linear","Poly","RF","SVR","KR"]

# Text style with boldface for the best configuration
StyleBestConfig = xlwt.easyxf('font: bold on')

# Create the workbook
MyWorkBook = xlwt.Workbook()

# Loop for all the tested performance measures (one per worksheet in the Excel spreadsheet file)
for MyPerformanceMeasure in TestedPerformanceMeasures:

  # Add a sheet for the current performance measure
  MyWorkSheet = MyWorkBook.add_sheet(MyPerformanceMeasure)

  # Create an array with the performance values for all sets and regressors
  MyPerformances=np.zeros((NumSets,len(TestedRegressors)))
  for NdxRegressor in range(0,len(TestedRegressors)):
    MyRegressor=TestedRegressors[NdxRegressor]
    # Obtain the performance values for all sets of input features
    for NdxSet in range(0,NumSets):
      MyPerformances[NdxSet,NdxRegressor]=ValidationResultsAverages[NdxSet,MyPerformanceMeasure,MyRegressor]

  # Obtain the indices of the best performing configuration (set and regressor)
  # for this performance measure
  if HigherBetterPerformance[MyPerformanceMeasure]:
    # Higher is better
    IndexBestConfig = np.unravel_index(np.argmax(MyPerformances, axis=None), (NumSets,len(TestedRegressors)))
  else:
    # Lower is better
    IndexBestConfig = np.unravel_index(np.argmin(MyPerformances, axis=None), (NumSets,len(TestedRegressors)))

  # Sort the sets by their performance. The performance of a set is the performance of the
  # best regressor for that set
  if HigherBetterPerformance[MyPerformanceMeasure]:
    # Higher is better
    BestRegressorPerformances=np.amax(MyPerformances, axis=1)
    RankedSetIndices=np.argsort(-BestRegressorPerformances)
  else:
    # Lower is better
    BestRegressorPerformances=np.amin(MyPerformances, axis=1)
    RankedSetIndices=np.argsort(BestRegressorPerformances)

  # Show a heatmap with the performance values
  HeatMapValues=np.zeros((CountInputFeatures,CountInputFeatures))
  # Index to traverse the BestRegressorPerformances array
  NdxThisFeature=0
  # Process all feature sets with a single feature
  for NdxFeature in range(0,CountInputFeatures):
    HeatMapValues[NdxFeature,NdxFeature]=BestRegressorPerformances[NdxThisFeature]
    NdxThisFeature=NdxThisFeature+1
  # Process all feature sets with two features
  for NdxFeature in range(0,CountInputFeatures):
    for NdxFeature2 in range(NdxFeature+1,CountInputFeatures):
      HeatMapValues[NdxFeature,NdxFeature2]=BestRegressorPerformances[NdxThisFeature]
      HeatMapValues[NdxFeature2,NdxFeature]=HeatMapValues[NdxFeature,NdxFeature2]
      NdxThisFeature=NdxThisFeature+1
  # MyPerformances[20,4]=np.nan # The KR for the 3 4 5 8 set works badly for bw="aic"
  MyFigure, MyAxes = plt.subplots()
  MyImage = MyAxes.imshow(HeatMapValues)
  # Show all ticks and label them with the respective list entries
  MyAxes.set_yticks([0,5,10,14])
  MyAxes.set_yticklabels(["0","5","10","14"])
  MyAxes.set_xticks([0,5,10,14])
  MyAxes.set_xticklabels(["0","5","10","14"])
  # Rotate the tick labels and set their alignment
  plt.setp(MyAxes.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
  heatmatp_path = os.path.join(result_path + '/%s.pdf' % MyPerformanceMeasure)
  plt.savefig(heatmatp_path, format="pdf", bbox_inches="tight")
  plt.show()


  # Generate a table with the performance values, sorted by the set indices
  # Write the first row which lists the regressors
  for NdxRegressor in range(0,len(TestedRegressors)):
    MyRegressor=TestedRegressors[NdxRegressor]
    MyWorkSheet.write(0,NdxRegressor+1,MyRegressor)
  # Loop for all sets (one set per row in the worksheet)
  for NdxSet in range(0,NumSets):
    # Write the model identification
    MyWorkSheet.write(NdxSet+1,0," ".join(map(str,SetsInputFeatures[NdxSet])))
    # Loop for all regressors (one regressor per column in the worksheet)
    for NdxRegressor in range(0,len(TestedRegressors)):
      MyRegressor=TestedRegressors[NdxRegressor]
      # Write the performance value. Apply boldface in case that it is the best configuration
      if (IndexBestConfig[0]==NdxSet) and (IndexBestConfig[1]==NdxRegressor):
        MyWorkSheet.write(NdxSet+1,NdxRegressor+1,ValidationResultsAverages[NdxSet,MyPerformanceMeasure,MyRegressor],StyleBestConfig)
      else:
        MyWorkSheet.write(NdxSet+1,NdxRegressor+1,ValidationResultsAverages[NdxSet,MyPerformanceMeasure,MyRegressor])

  # Generate a table with the performance values, sorted by the performance of the sets
  # Write the first row which lists the regressors
  for NdxRegressor in range(0,len(TestedRegressors)):
    MyRegressor=TestedRegressors[NdxRegressor]
    MyWorkSheet.write(0,NdxRegressor+len(TestedRegressors)+3,MyRegressor)
  # Loop for all sets (one set per row in the worksheet)
  for NdxSet in range(0,NumSets):
    # The index of the NdxSet-th best set according to their performance
    NdxMyRankedSet=RankedSetIndices[NdxSet]
    # Write the model identification
    MyWorkSheet.write(NdxSet+1,len(TestedRegressors)+2," ".join(map(str,SetsInputFeatures[NdxMyRankedSet])))
    # Loop for all regressors (one regressor per column in the worksheet)
    for NdxRegressor in range(0,len(TestedRegressors)):
      MyRegressor=TestedRegressors[NdxRegressor]
      # Write the performance value. Apply boldface in case that it is the best configuration
      if (IndexBestConfig[0]==NdxMyRankedSet) and (IndexBestConfig[1]==NdxRegressor):
        MyWorkSheet.write(NdxSet+1,NdxRegressor+len(TestedRegressors)+3,ValidationResultsAverages[NdxMyRankedSet,MyPerformanceMeasure,MyRegressor],StyleBestConfig)
      else:
        MyWorkSheet.write(NdxSet+1,NdxRegressor+len(TestedRegressors)+3,ValidationResultsAverages[NdxMyRankedSet,MyPerformanceMeasure,MyRegressor])
  
# Save the Excel spreadsheet file
excel_path = os.path.join(result_path + '/CrossValidationResults.xls')
MyWorkBook.save(excel_path)